# Coursera recomender System

## Imports

In [115]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1.Gathering Data from Coursera via request API

In [109]:
import pandas as pd
import requests

def get_courses_data():
    api_url = "https://api.coursera.org/api/courses.v1"
    params = {
        "start": 0,
        "limit": 10000,
        "fields": "id,name,slug,startDate,primaryLanguages,workload,isPartOfSpecialization"
    }
    response = requests.get(api_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        courses = data["elements"]
        
        course_data = []
        for course in courses:
            course_id = course["id"]
            course_name = course["name"]
            course_slug = course["slug"]
            start_date = course.get("startDate", "")
            end_date = course.get("endDate", "")
            primary_languages = course.get("primaryLanguages", [])
            workload = course.get("workload", "")
            primary_category = course.get("primaryCategory", "")
            instructor_ids = course.get("instructorIds", [])
            is_part_of_specialization = course.get("isPartOfSpecialization", False)
            skills = course.get("skills", [])
            audience = course.get("audience", "")
            course_format = course.get("courseFormat", "")
            
            course_data.append({
                "ID": course_id,
                "Name": course_name,
                "Slug": course_slug,
                "Start Date": start_date,
                "Primary Languages": primary_languages[0].split('-')[0],
                "Workload": workload,
                #"primary_category":primary_category,
                #"Instructor IDs": instructor_ids,
                "Is Part of Specialization": is_part_of_specialization,
            })
        
        df = pd.DataFrame(course_data)
        return df
    else:
        print("Failed to retrieve data. Status code:", response.status_code)
        return None

courses_df = get_courses_data()


,ID,Name,Slug,Start Date,Primary Languages,Workload,Is Part of Specialization
0,NJSdGN71Eeq4CApSN3OTvQ,Make Your Pick-Ups Look Cool in Unity (Intro t...,make-pick-ups-look-cool-unity-introduction-ani...,1598035773172,en,2 hours,False
1,DMkcgX7LEeyRTg6FtAvfBw,Intégrer des applications dans votre Dashboard...,integrer-applications-dashboard-hootsuite,1644614698077,fr,2 heures,False
2,YLO0oGSUEeyIUg4Qv2RsBQ,كيفية استعمال التطبيق هووتسويت,getting-started-with-hootsuite-ar,1642193091376,ar,2 hours,False
3,69Bku0KoEeWZtA4u62x6lQ,Gamification,gamification,1447095621493,en,4-8 hours/week,False
4,0HiU7Oe4EeWTAQ4yevf_oQ,Dealing With Missing Data,missing-data,1471901099812,en,"4 weeks of study, 1-2 hours/week",False
...,...,...,...,...,...,...,...
9995,ng-bUg0kEe2Y4woRe4onvw,Bildklassifizierung mit Tensorflow,bildklassifizierung-mit-tensorflow,1659393633053,de,1 Stunde,False
9996,oRWiSiO-EeuZCg52ZqAIxQ,الاستراتيجية التنافسية,competitive-strategy-ar,1619536132986,ar,10 ساعات فيديو واختبارات,False
9997,iClli1IjEey1wAq1DMRz-Q,Intro to Operating Systems 4: Persistence,codio-intro-to-operating-systems-4-persistence,1641316357195,en,,False
9998,a19TzyN0Eeu-rg7jvs9-1w,Bases de la planification et de la gestion de ...,uva-darden-project-management-fr,1618419430243,fr,"4 semaines d’études, 2-3 heures/semaine",False


In [114]:
courses_df['Start Date'] = pd.to_datetime(courses_df['Start Date'])
courses_df['Start Date'] = courses_df['Start Date'].dt.strftime('%d-%m-%Y %H:%M')
courses_df['Start Date'].describe()

count                10000
unique                   7
top       01-01-1970 00:27
freq                  4181
Name: Start Date, dtype: object

## 2.Database

### 2-1.Save to DB

In [8]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

def save_df_to_mysql(df, database, table_name, host='localhost', user='your_username', password='your_password'):
    # Create a MySQL connection
    connection = pymysql.connect(
        host=host,
        user=user,
        password=password
    )
    
    # Create the database if it doesn't exist
    cursor = connection.cursor()
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database}")
    cursor.close()
    
    # Create SQLAlchemy engine
    engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")
    
    # Save the DataFrame to the MySQL table
    df.to_sql(table_name, engine, if_exists='replace', index=False)
    
    # Close the connection
    connection.close()
    print(f"DataFrame saved to the '{database}.{table_name}' table.")

In [9]:
database_name = "DBLAB_final_project"
table_name = "test3"
save_df_to_mysql(courses_df, database_name, table_name, host='localhost', user='root', password='pdne')

DataFrame saved to the 'DBLAB_final_project.test3' table.


### 2-2.Connect to DB

In [1]:
import mysql
import mysql.connector
# run this every time
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="pdne",
 
)
mycursor = mydb.cursor()

mycursor.execute("SHOW DATABASES")
for db in mycursor:
    print(db)

('Coursera',)
('Coursera_DBLAB',)
('DB_Lab',)
('DBLAB_final_project',)
('information_schema',)
('myDatabase',)
('mysql',)
('performance_schema',)
('Session_Data',)
('Study',)
('sys',)
('TopLearn_Project',)


In [5]:
import pandas as pd
mycursor.execute("USE DBLAB_final_project")
courses_df = pd.read_sql('SELECT * FROM test3', mydb)
courses_df

/var/folders/bf/9616ns414yg2qn2p8czl2sr00000gn/T/ipykernel_18149/967033774.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  courses_df = pd.read_sql('SELECT * FROM test3', mydb)


,ID,Name,Slug,Start Date,Primary Languages,Workload,Is Part of Specialization
0,NJSdGN71Eeq4CApSN3OTvQ,Make Your Pick-Ups Look Cool in Unity (Intro t...,make-pick-ups-look-cool-unity-introduction-ani...,1598035773172,en,2 hours,0
1,DMkcgX7LEeyRTg6FtAvfBw,Intégrer des applications dans votre Dashboard...,integrer-applications-dashboard-hootsuite,1644614698077,fr,2 heures,0
2,YLO0oGSUEeyIUg4Qv2RsBQ,كيفية استعمال التطبيق هووتسويت,getting-started-with-hootsuite-ar,1642193091376,ar,2 hours,0
3,69Bku0KoEeWZtA4u62x6lQ,Gamification,gamification,1447095621493,en,4-8 hours/week,0
4,0HiU7Oe4EeWTAQ4yevf_oQ,Dealing With Missing Data,missing-data,1471901099812,en,"4 weeks of study, 1-2 hours/week",0
...,...,...,...,...,...,...,...
95,XXZBGc97EeufchLeGgZGZQ,Relational Database Administration (DBA),relational-database-administration,1637617419532,en,"5 weeks of study, 2-4 hours / week",0
96,M1lJ0BrjEeui5Apx-DHBkQ,Waits in Selenium Test Automation Tool,waits-in-selenium-test-automation-tool,1604094399504,en,2 hours,0
97,LZZg6vhQEeWfYgqbi1xsdw,Chinese Characters for beginner 汉字,hanzi,1477679733728,zh,5-7 hours/week,0
98,wFc3FhwIEeyf4Ar-aygVCw,Introducción a Hootsuite,introduccion-a-hootsuite,1633635891410,es,2 hours,0


## 3.Dataframe and Analysis

In [116]:
data = pd.read_csv("Coursera.csv")
data.head(5)

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills
0,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,4.8,https://www.coursera.org/learn/write-a-feature...,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,4.8,https://www.coursera.org/learn/canvas-analysis...,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,�cole Polytechnique,Advanced,4.1,https://www.coursera.org/learn/silicon-thin-fi...,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Finance for Managers,IESE Business School,Intermediate,4.8,https://www.coursera.org/learn/operational-fin...,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,4.6,https://www.coursera.org/learn/single-table-sq...,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...


In [117]:
data.shape

(3522, 7)

In [118]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3522 entries, 0 to 3521
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Course Name         3522 non-null   object
 1   University          3522 non-null   object
 2   Difficulty Level    3522 non-null   object
 3   Course Rating       3522 non-null   object
 4   Course URL          3522 non-null   object
 5   Course Description  3522 non-null   object
 6   Skills              3522 non-null   object
dtypes: object(7)
memory usage: 192.7+ KB


In [119]:
data.isnull().sum()

Course Name           0
University            0
Difficulty Level      0
Course Rating         0
Course URL            0
Course Description    0
Skills                0
dtype: int64

In [120]:
data['Difficulty Level'].value_counts()

Beginner          1444
Advanced          1005
Intermediate       837
Conversant         186
Not Calibrated      50
Name: Difficulty Level, dtype: int64

In [121]:
data['Course Rating'].value_counts()

4.7               740
4.6               623
4.8               598
4.5               389
4.4               242
4.9               180
4.3               165
4.2               121
5                  90
4.1                85
Not Calibrated     82
4                  51
3.8                24
3.9                20
3.6                18
3.7                18
3.5                17
3.4                13
3                  12
3.2                 9
3.3                 6
2.9                 6
2.6                 2
2.8                 2
2.4                 2
1                   2
2                   1
3.1                 1
2.5                 1
1.9                 1
2.3                 1
Name: Course Rating, dtype: int64

In [122]:
data['University'].value_counts()

Coursera Project Network                      562
University of Illinois at Urbana-Champaign    138
Johns Hopkins University                      110
University of Michigan                        101
University of Colorado Boulder                101
                                             ... 
GitLab                                          1
Yeshiva University                              1
University of Glasgow                           1
Laureate Education                              1
The World Bank Group                            1
Name: University, Length: 184, dtype: int64

In [123]:
data['Skills'].value_counts()

Google Cloud Platform  Big Data  Cloud Infrastructure  google storage  Cloud Storage  bigquery  Cloud Platforms  Kubernetes  Cloud Computing  Google App Engine information-technology cloud-computing                                  8
art  listening  emotions  Sound Design  music  signature  Audio Production  Writing  microphone  sound arts-and-humanities music-and-art                                                                                                4
Python Programming  python libraries  analysis  semantics  Computer Programming  syntax  Data Analysis  Pandas  python syntax and semantics  Numpy data-science data-analysis                                                           4
analysis  Machine Learning  Data Mining  Artificial Neural Networks  Regression Analysis  Data Analysis  data reporting  Human Learning  Regression  Big Data data-science data-analysis                                                4
Machine Learning  artificial general intelligence  application d

In [124]:
data['Course Name']

0       Write A Feature Length Screenplay For Film Or ...
1       Business Strategy: Business Model Canvas Analy...
2                           Silicon Thin Film Solar Cells
3                                    Finance for Managers
4            Retrieve Data using Single-Table SQL Queries
                              ...                        
3517    Capstone: Retrieving, Processing, and Visualiz...
3518                     Patrick Henry: Forgotten Founder
3519    Business intelligence and data analytics: Gene...
3520                                  Rigid Body Dynamics
3521    Architecting with Google Kubernetes Engine: Pr...
Name: Course Name, Length: 3522, dtype: object

## Required Columns for System

#### Important columns to be used in recommendation system : 

- Course Ratings : Users may want to see courses based on rates
- Skills : Users may want to see courses based on same skills
- Difficulty Level : Similar courses as per difficulty level 

#### Columns not used for the recommendation system :

- Course Name : Names of the courses
- Course Description : Similar courses may have similar course description but not useful for us
- University : Same university might offer multiple courses in different domains which the user might not want to see
- Course URL : No significance in the recommendation system

In [125]:
data = data[['Course Name','Difficulty Level','Course Rating','Skills']]
data.head(5)

,Course Name,Difficulty Level,Course Rating,Skills
0,Write A Feature Length Screenplay For Film Or ...,Beginner,4.8,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Beginner,4.8,Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,Advanced,4.1,chemistry physics Solar Energy film lambda...
3,Finance for Managers,Intermediate,4.8,accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Beginner,4.6,Data Analysis select (sql) database manageme...


## 4.GUI environment

In [7]:
import pandas as pd
import tkinter as tk
from tkinter import ttk

# Assuming you have the 'courses_df' DataFrame

# Create a Tkinter window
window = tk.Tk()

# Get the first 10 rows of the DataFrame
first_10_rows = courses_df.head(10)

# Create a Treeview widget to display the table
table = ttk.Treeview(window)

# Configure the table columns based on the DataFrame columns
table["columns"] = list(first_10_rows.columns)
for column in first_10_rows.columns:
    table.column(column, width=100)
    table.heading(column, text=column)

# Insert the data rows into the table
for _, row in first_10_rows.iterrows():
    table.insert("", "end", values=tuple(row))

# Place the table in the window
table.pack()

# Start the Tkinter event loop
window.mainloop()
